In [1]:
from sys import path
from os import getcwd, environ

path.insert(0, getcwd())
path.insert(0, getcwd() + "/modules")
from modules.DataMod import DataSet
from modules.CustomLosses import LSSIM
from modules.misc import ssim_metric, psnrb_metric
from modules.ImageMetrics.metrics import three_ssim
from tensorflow.keras.optimizers import Adam

import mlflow.keras

from models.autoEncoder import autoEncoder
from models.ResidualAutoencoder import residualAutoEncoder
from models.Unet import unet

from multiprocessing import Process
from datetime import datetime

environ["CUDA_VISIBLE_DEVICES"] = "1,3"

2023-07-23 21:43:02.316138: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 21:43:02.398652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 21:43:02.399756: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 21:43:04.388616: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
637501
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
32748
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Fetching Datasets

In [ ]:
# creates the datasets
tinyDataSet, cifarDataSet, cifarAndTinyDataSet = DataSet(), DataSet(), DataSet()

tinyDataSet = tinyDataSet.load_rafael_tinyImagenet_64x64_noise_data()
cifarDataSet = cifarDataSet.load_rafael_cifar_10_noise_data()

# concatenates the datasets
cifarAndTinyDataSet = cifarAndTinyDataSet.concatenateDataSets(cifarDataSet, tinyDataSet)

## Training Models

In [ ]:
# to do: 
# paralelize the training (does it's necessary?)
# batch size shoudn't be specified (keras API doc), does it affect the training?

# training with LSSIM loss function and ssim and psnrb metrics


In [ ]:
# compiles all the models

autoEncoder.compile(optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False), loss = LSSIM(), metrics = [ssim_metric, psnrb_metric, three_ssim])
unet.compile(optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False), loss = LSSIM(), metrics = [ssim_metric, psnrb_metric, three_ssim])
residualAutoEncoder.compile(optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, amsgrad=False), loss = LSSIM(), metrics = [ssim_metric, psnrb_metric, three_ssim])

In [3]:
batch_size = 20
epochs = 15
# how to decide the number of epochs and batch size?

mlflow.keras.autolog()

# function to parallelize the training
def train_model_paralel(model):
        # trains the models with the datasets
        print(model.name)
        for dataset in [tinyDataSet, cifarDataSet, cifarAndTinyDataSet]:
                with mlflow.start_run(run_name= model.name +dataset.name):

                        history = model.fit(
                                x = dataset.x_train,
                                y = dataset.y_train,
                                batch_size = batch_size,
                                epochs = epochs,
                                verbose = 1,
                                validation_split = 0,
                                shuffle = True,
                                class_weight = None,
                                sample_weight = None,
                                steps_per_epoch = None,
                                validation_steps = None,
                                validation_batch_size = None,
                                validation_freq = 1,
                                max_queue_size = 10,
                                workers = 1,
                                use_multiprocessing = False
                        )

                        score = model.evaluate(dataset.x_test, dataset.y_test, verbose = 1)

                        try:
                                model.save_weights("models/weights/run1/" + model.name + dataset.name + ".h5")
                        except:
                                print("Error saving weights")

In [4]:
start_date = datetime.now()

procs = []

# launches the training in parallel
for model in [autoEncoder, unet, residualAutoEncoder]:#, unet, residualAutoEncoder]:
        proc = Process(target=train_model_paralel, args=(model, ))
        proc.start()
        procs.append(proc)

# waits for the training to finish
for proc in procs:
        proc.join()

end_date = datetime.now()

AutoEncoder-2.3-64x64


Process Process-1:


Unet2.3-64x64

Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()


  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-2:
  File "/tmp/ipykernel_7027/3828286023.py", line 11, in train_model_paralel
    for dataset in [tinyDataSet, cifarDataSet, cifarAndTinyDataSet]:


ResidualAutoEncoder-0.1-64x64

Traceback (most recent call last):
NameError: name 'tinyDataSet' is not defined
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()


  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-3:
  File "/tmp/ipykernel_7027/3828286023.py", line 11, in train_model_paralel
    for dataset in [tinyDataSet, cifarDataSet, cifarAndTinyDataSet]:
NameError: name 'tinyDataSet' is not defined
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_7027/3828286023.py", line 11, in train_model_paralel
    for dataset in [tinyDataSet, cifarDataSet, cifarAndTinyDataSet]:
NameError: name 'tinyDataSet' is not defined


In [ ]:
with open("logs/run1.txt", "w") as file:
        file.write("Start date: " + str(start_date) + "\n")
        file.write("End date: " + str(end_date) + "\n")
        file.write("Duration: " + str(end_date - start_date) + "\n")